<h1>Applied Data Science Capstone Project - Week 5</h1>

## 1. Introduction

Bakery industry is a $3.5 Billion USD (2017) industry in Canada. It is expected to  continue to grow by a compound annualized rate of 3.4%. While it is a completive market with moderate growth, with passion and skill, it is a great industry to be in. 

Celeste has just finished culinary school and graduated top of her class from Le Cordon Bleu in Paris, France. She is going back home to Toronto, Canada. Her proud parents have secured funding to invest in her dream French patisserie. Her ideal location would be in a place that has other businesses around but not too many other bakeries in downtown Toronto area.


## 2. Data

<h3>Import Necessary Libraries</h3>

- List of postal codse, neighborhoods, and burough’s in Toronto.
- Latitude and longitide coordinates of those neighborhoods.
- Foursquare location date which contains venues in each of the neighborhoods.
  This data will be used to perform clustering to determine if a certain neighborhood is suitable for Celeste’s business.

In [1]:
!pip install geopy
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

from geopy.geocoders import Nominatim 

from pandas.io.json import json_normalize 


import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

! pip install BeautifulSoup4
from bs4 import BeautifulSoup

!conda install -c conda-forge folium=0.5.0 --yes 
import folium

import urllib.request


     |████████████████████████████████| 112kB 23.7MB/s eta 0:00:01
     |████████████████████████████████| 122kB 6.1MB/s eta 0:00:01
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.5
  latest version: 4.9.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    brotlipy-0.7.0             |py36he6145b8_1001         347 KB  conda-forge
    certifi-2020.6.20          |   py36h9880bd3_2         151 KB  cond

<h3>Load Data</h3>

In [2]:
src_data_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

req = urllib.request.urlopen(src_data_url)
article = req.read().decode()

with open('List_of_postal_codes_of_Canada:_M', 'w') as fo:
    fo.write(article)

In [3]:
article = open('List_of_postal_codes_of_Canada:_M').read()
soup = BeautifulSoup(article, 'html.parser')
tables = soup.find_all('table', class_='sortable')

for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings[:3] == ['Postcode', 'Borough', 'Neighbourhood']:
        break

In [4]:
data = []
for tr in table.find_all('tr'):
    tds = tr.find_all('td')
    if not tds:
        continue
    code,borough,neighbourhood = [td.text.strip() for td in tds[:3]]
    if 'Not assigned' in borough:
        continue
    if  'Not assigned' in neighbourhood:
        neighbourhood = borough
            
    data.append([code,borough,neighbourhood])

In [5]:
temp_data = np.array(data)
temp_df = pd.DataFrame({'Postcode':temp_data[:,0],'Borough':temp_data[:,1],'Neighbourhood':temp_data[:,2]})
temp_df['is_duplicated'] = temp_df.duplicated(['Postcode'])
tt = temp_df[(temp_df['is_duplicated'] == True)]

tt.drop('is_duplicated',axis=1,inplace=True)
temp_df.drop('is_duplicated',axis=1,inplace=True)
temp_df.drop_duplicates('Postcode',keep='first',inplace=True)
temp_df.set_index('Postcode', inplace=True)

for row in tt.itertuples():
    if temp_df.loc[row.Postcode] is not "":
        if row.Neighbourhood not in temp_df.loc[row.Postcode].Neighbourhood:
            temp_df.loc[row.Postcode].Neighbourhood = temp_df.loc[row.Postcode].Neighbourhood + ',' + row.Neighbourhood

        
print(temp_df.shape)

(103, 2)


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inplace=inplace,


In [6]:
csv_file = 'Toronto_PO_Codes.csv'
print('Saving data to %s' % csv_file)
temp_df.to_csv(csv_file,sep=';', encoding='utf-8',index=True)

Saving data to Toronto_PO_Codes.csv


In [41]:
df_Toronto = pd.read_csv(csv_file,sep=';')
df_Toronto.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
df_Toronto.shape

(103, 3)

In [43]:
#Ignore cells with a borough that is Not assigned.'
df_Toronto.drop(df_Toronto.index[df_Toronto['Borough'] == 'Not assigned'], inplace = True)
df_Toronto = df_Toronto.reset_index(drop=True)
df_Toronto.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [45]:
df_Toronto.loc[df_Toronto['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df_Toronto['Borough']
df_Toronto.shape

(103, 3)

In [46]:
df_coords = pd.read_csv('Geospatial_Coordinates.csv')   
df_coords.rename(columns={'Postal Code':'Postcode'},inplace=True)
df_coords.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [70]:
toronto_data ['Latitude']=df_coords['Latitude'].values
toronto_data ['Longitude']=df_coords['Longitude'].values
toronto_data 

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.744734,-79.239476
6,M1B,Scarborough,"Malvern, Rouge",43.727929,-79.262029
7,M3B,North York,Don Mills,43.711112,-79.284577
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.716316,-79.239476
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848


Prepare the clustering of neighbourhoods

In [73]:
TRT_data = toronto_data[toronto_data['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
TRT_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848
3,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389
4,M5E,Downtown Toronto,Berczy Park,43.757490,-79.374714


In [48]:
address = 'Downtown Toronto, ON'
geolocator = Nominatim(user_agent="trt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [54]:
# create map of Downtown Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [19]:
CLIENT_ID = 'WDFMNEIE4IRRW3K20NBLS0SVKWEXIWYGSDVCHSHETEDLY3S0'
CLIENT_SECRET = 'QLF1ITON4QK4ODQ04KNNOMUKORTB10MBRKUZ1XZ4KGVL0ERG'
VERSION = '20180605' 
LIMIT=500

In [59]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] 
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] 

neighborhood_name = toronto_data.loc[0, 'Neighbourhood']
LIMIT=100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, neighborhood_latitude, 
    neighborhood_longitude, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=WDFMNEIE4IRRW3K20NBLS0SVKWEXIWYGSDVCHSHETEDLY3S0&client_secret=QLF1ITON4QK4ODQ04KNNOMUKORTB10MBRKUZ1XZ4KGVL0ERG&ll=43.6563221,-79.3809161&v=20180605&query=43.7532586&radius=-79.3296565&limit=500'

In [64]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [74]:
toronto_venues = getNearbyVenues(names=TRT_data['Neighbourhood'],
                                   latitudes=TRT_data['Latitude'],
                                   longitudes=TRT_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [75]:
print(toronto_venues.shape)
toronto_venues.head()

(151, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
1,"Regent Park, Harbourfront",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
2,"Regent Park, Harbourfront",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant
3,"Regent Park, Harbourfront",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
4,"Regent Park, Harbourfront",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location


In [77]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,1,1,1,1,1,1
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,5,5,5,5,5,5
Christie,2,2,2,2,2,2
Church and Wellesley,6,6,6,6,6,6
"Commerce Court, Victoria Hotel",2,2,2,2,2,2
"First Canadian Place, Underground city",1,1,1,1,1,1
"Garden District, Ryerson",4,4,4,4,4,4
"Harbourfront East, Union Station, Toronto Islands",7,7,7,7,7,7


In [78]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 85 uniques categories.


In [80]:
toronto_venues['Venue Category'].unique()[:50]

array(['Bank', 'Electronics Store', 'Restaurant', 'Mexican Restaurant',
       'Rental Car Location', 'Medical Center', 'Intersection',
       'Breakfast Spot', 'Caribbean Restaurant', 'Hakka Restaurant',
       'Thai Restaurant', 'Athletics & Sports', 'Bakery', 'Gas Station',
       'Fried Chicken Joint', 'Café', 'General Entertainment',
       'Skating Rink', 'College Stadium', 'Chinese Restaurant',
       'Grocery Store', 'Sandwich Place', 'Fast Food Restaurant',
       'Pharmacy', 'Coffee Shop', 'Pizza Place', 'Gym',
       'Indian Restaurant', 'Martial Arts School', 'Butcher', 'Park',
       'Food & Drink Shop', 'Airport', 'Curling Ice', 'Beer Store',
       'Dance Studio', 'Ice Cream Shop', 'Fish & Chips Shop',
       'Sushi Restaurant', 'Brewery', 'Liquor Store',
       'Italian Restaurant', 'Pub', 'Pet Store', 'Steakhouse',
       'Burrito Place', 'Movie Theater', 'Trail', 'Playground',
       'Discount Store'], dtype=object)

In [82]:
# check if the results contain "Bakery"
"Bakery" in toronto_venues['Venue Category'].unique()

True

Analyze Each Neighborhood

In [84]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Airport,Athletics & Sports,Auto Workshop,Bakery,Bank,Bar,Baseball Field,Beer Store,Bookstore,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Comic Shop,Construction & Landscaping,Convenience Store,Curling Ice,Dance Studio,Dessert Shop,Diner,Discount Store,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,General Entertainment,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Hardware Store,Health Food Store,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Kids Store,Latin American Restaurant,Light Rail Station,Liquor Store,Martial Arts School,Medical Center,Mexican Restaurant,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pub,Recording Studio,Rental Car Location,Restaurant,Sandwich Place,Skate Park,Skating Rink,Smoothie Shop,Social Club,Steakhouse,Supplement Shop,Sushi Restaurant,Tanning Salon,Thai Restaurant,Thrift / Vintage Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Wings Joint,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [85]:
toronto_onehot.shape

(151, 86)

In [86]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Airport,Athletics & Sports,Auto Workshop,Bakery,Bank,Bar,Baseball Field,Beer Store,Bookstore,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Comic Shop,Construction & Landscaping,Convenience Store,Curling Ice,Dance Studio,Dessert Shop,Diner,Discount Store,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,General Entertainment,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Hardware Store,Health Food Store,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Kids Store,Latin American Restaurant,Light Rail Station,Liquor Store,Martial Arts School,Medical Center,Mexican Restaurant,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pub,Recording Studio,Rental Car Location,Restaurant,Sandwich Place,Skate Park,Skating Rink,Smoothie Shop,Social Club,Steakhouse,Supplement Shop,Sushi Restaurant,Tanning Salon,Thai Restaurant,Thrift / Vintage Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Wings Joint,Yoga Studio
0,Berczy Park,0.0,0.000000,0.0000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000,0.000000,0.000000,0.00,0.0000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.000,0.0,0.0000,0.0000,0.000,0.00,0.000000,0.000000,0.000000,0.0000,0.000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,1.0,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.0000,0.000,0.0000,0.0,0.0,0.000000,0.0000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.000000,0.0625,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.062500,0.0000,0.062500,0.0625,0.000000,0.000,0.000000,0.000000,0.00,0.0625,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0625,0.062500,0.000000,0.0,0.000000,0.000,0.0,0.0625,0.0625,0.000,0.00,0.000000,0.000000,0.000000,0.0625,0.000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0625,0.000000,0.0,0.000,0.000,0.000000,0.062500,0.000000,0.000000,0.062500,0.0,0.000000,0.0625,0.000,0.062500,0.000000,0.0625,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.0000,0.000,0.0000,0.0,0.0,0.000000,0.0000,0.000000
2,Central Bay Street,0.0,0.000000,0.0000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.2000,0.000000,0.000,0.000000,0.200000,0.00,0.0000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.000,0.0,0.0000,0.0000,0.000,0.00,0.000000,0.200000,0.000000,0.0000,0.000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.0,0.000,0.000,0.000000,0.000000,0.000000,0.200000,0.200000,0.0,0.000000,0.0000,0.000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.0000,0.000,0.0000,0.0,0.0,0.000000,0.0000,0.000000
3,Christie,0.0,0.000000,0.0000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000,0.000000,0.000000,0.00,0.0000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0000,0.000000,0.000000,0.5,0.000000,0.000,0.0,0.0000,0.0000,0.000,0.00,0.000000,0.000000,0.000000,0.0000,0.000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.0,0.000,0.000,0.000000,0.500000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.0000,0.000,0.0000,0.0,0.0,0.000000,0.0000,0.000000
4,Church and Wellesley,0.0,0.000000,0.0000,0.0000,0.000000,0.000000,0.0,0.0000

In [87]:
len(toronto_grouped[toronto_grouped["Bakery"] > 0])

2

Create a new DataFrame for Bakery data only

In [120]:
trt1_mall = toronto_grouped[["Neighborhood","Bakery"]]

trt1_mall.head()

,Neighborhood,Bakery
0,Berczy Park,0.0
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.0
2,Central Bay Street,0.0
3,Christie,0.0
4,Church and Wellesley,0.0


Cluster neighborhoods

In [93]:
kclusters = 3
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10]

array([1, 0, 0, 0, 0, 0, 2, 0, 0, 0], dtype=int32)

In [94]:
neighbourhoods_venues_sorted.head()

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Agincourt,Clothing Store,Lounge,Breakfast Spot,Skating Rink,Latin American Restaurant,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
1,1,"Alderwood, Long Branch",Pizza Place,Sandwich Place,Coffee Shop,Pub,Pharmacy,Gym,Greek Restaurant,Discount Store,Department Store,Dessert Shop
2,1,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pharmacy,Ice Cream Shop,Bridal Shop,Shopping Mall,Sandwich Place,Diner,Middle Eastern Restaurant,Restaurant
3,1,Bayview Village,Japanese Restaurant,Café,Chinese Restaurant,Bank,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
4,1,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop,Greek Restaurant,Thai Restaurant,Locksmith,Liquor Store,Comfort Food Restaurant,Juice Bar,Butcher


In [95]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
trt1_merged = trt1_mall.copy()

# add clustering labels
trt1_merged["Cluster Labels"] = kmeans.labels_

In [96]:
trt1_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
trt1_merged.head()

,Neighborhood,Bakery,Cluster Labels
0,Berczy Park,0.0,1
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0
2,Central Bay Street,0.0,0
3,Christie,0.0,0
4,Church and Wellesley,0.0,0


In [99]:
trt1_merged = trt1_merged.join(TRT_data.set_index("Neighbourhood"), on="Neighborhood")

print(trt1_merged.shape)
trt1_merged # check the last columns!

(19, 7)


,Neighborhood,Bakery,Cluster Labels,Postcode,Borough,Latitude,Longitude
0,Berczy Park,0.0000,1,M5E,Downtown Toronto,43.757490,-79.374714
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.0000,0,M5V,Downtown Toronto,43.662744,-79.321558
2,Central Bay Street,0.0000,0,M5G,Downtown Toronto,43.782736,-79.442259
3,Christie,0.0000,0,M6G,Downtown Toronto,43.753259,-79.329656
4,Church and Wellesley,0.0000,0,M4Y,Downtown Toronto,43.696319,-79.532242
5,"Commerce Court, Victoria Hotel",0.0000,0,M5L,Downtown Toronto,43.689574,-79.383160
6,"First Canadian Place, Underground city",0.0000,2,M5X,Downtown Toronto,43.724766,-79.532242
7,"Garden District, Ryerson",0.0000,0,M5B,Downtown Toronto,43.692657,-79.264848
8,"Harbourfront East, Union Station, Toronto Islands",0.0000,0,M5J,Downtown Toronto,43.695344,-79.318389
9,"Kensington Market, Chinatown, Grange Park",0.0000,0,M5T,Downtown Toronto,43.651571,-79.484450


In [100]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(trt1_merged['Latitude'], trt1_merged['Longitude'], trt1_merged['Neighborhood'], trt1_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine Clusters

Cluster 0

In [102]:
trt1_merged.loc[trt1_merged['Cluster Labels'] == 0]

,Neighborhood,Bakery,Cluster Labels,Postcode,Borough,Latitude,Longitude
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.0000,0,M5V,Downtown Toronto,43.662744,-79.321558
2,Central Bay Street,0.0000,0,M5G,Downtown Toronto,43.782736,-79.442259
3,Christie,0.0000,0,M6G,Downtown Toronto,43.753259,-79.329656
4,Church and Wellesley,0.0000,0,M4Y,Downtown Toronto,43.696319,-79.532242
5,"Commerce Court, Victoria Hotel",0.0000,0,M5L,Downtown Toronto,43.689574,-79.383160
7,"Garden District, Ryerson",0.0000,0,M5B,Downtown Toronto,43.692657,-79.264848
8,"Harbourfront East, Union Station, Toronto Islands",0.0000,0,M5J,Downtown Toronto,43.695344,-79.318389
9,"Kensington Market, Chinatown, Grange Park",0.0000,0,M5T,Downtown Toronto,43.651571,-79.484450
10,"Queen's Park, Ontario Provincial Government",0.1250,0,M7A,Downtown Toronto,43.773136,-79.239476
11,"Regent Park, Harbourfront",0.0000,0,M5A,Downtown Toronto,43.763573,-79.188711


Cluster 1

In [103]:
trt1_merged.loc[trt1_merged['Cluster Labels'] == 1]

,Neighborhood,Bakery,Cluster Labels,Postcode,Borough,Latitude,Longitude
0,Berczy Park,0.0,1,M5E,Downtown Toronto,43.75749,-79.374714


Cluster 2

In [104]:
trt1_merged.loc[trt1_merged['Cluster Labels'] == 2]

,Neighborhood,Bakery,Cluster Labels,Postcode,Borough,Latitude,Longitude
6,"First Canadian Place, Underground city",0.0,2,M5X,Downtown Toronto,43.724766,-79.532242
13,Rosedale,0.0,2,M4W,Downtown Toronto,43.636258,-79.498509


Checking out TOP 10 poplular venues in Downtown Toronto

In [107]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Martial Arts School,Yoga Studio,Fast Food Restaurant,Dance Studio,Dessert Shop,Diner,Discount Store,Electronics Store,Falafel Restaurant,Farmers Market
1,"CN Tower, King and Spadina, Railway Lands, Har...",Garden Center,Brewery,Light Rail Station,Comic Shop,Park,Pizza Place,Recording Studio,Restaurant,Butcher,Burrito Place
2,Central Bay Street,Pharmacy,Butcher,Pizza Place,Coffee Shop,Grocery Store,Bank,Fish & Chips Shop,Dessert Shop,Diner,Discount Store
3,Christie,Park,Food & Drink Shop,Yoga Studio,Fast Food Restaurant,Dance Studio,Dessert Shop,Diner,Discount Store,Electronics Store,Falafel Restaurant
4,Church and Wellesley,Pizza Place,Coffee Shop,Chinese Restaurant,Discount Store,Sandwich Place,Intersection,Yoga Studio,Falafel Restaurant,Dance Studio,Dessert Shop


In [117]:
trt_data = neighborhoods_venues_sorted[neighborhoods_venues_sorted['6th Most Common Venue'] == 'Dessert Shop'].reset_index(drop=True)
trt_data.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Christie,Park,Food & Drink Shop,Yoga Studio,Fast Food Restaurant,Dance Studio,Dessert Shop,Diner,Discount Store,Electronics Store,Falafel Restaurant
1,"St. James Town, Cabbagetown",Pizza Place,Furniture / Home Store,Fast Food Restaurant,Curling Ice,Dance Studio,Dessert Shop,Diner,Discount Store,Electronics Store,Falafel Restaurant


In [118]:
trt_data = neighborhoods_venues_sorted[neighborhoods_venues_sorted['3rd Most Common Venue'] == 'Bakery'].reset_index(drop=True)
trt_data.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Queen's Park, Ontario Provincial Government",Athletics & Sports,Gas Station,Bakery,Bank,Thai Restaurant,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Yoga Studio,Farmers Market


In [129]:
trt_data = neighborhoods_venues_sorted[neighborhoods_venues_sorted['2nd Most Common Venue'] == 'Coffee Shop'].reset_index(drop=True)
trt_data.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Church and Wellesley,Pizza Place,Coffee Shop,Chinese Restaurant,Discount Store,Sandwich Place,Intersection,Yoga Studio,Falafel Restaurant,Dance Studio,Dessert Shop
1,"Kensington Market, Chinatown, Grange Park",Café,Coffee Shop,Pub,Italian Restaurant,Sushi Restaurant,Pizza Place,Fish & Chips Shop,Latin American Restaurant,Indie Movie Theater,Health Food Store


## 3.Conclusion

In this project, we have gone through the process of identifying the business problem,
specifying the data required, extracting and preparing the data, performing machine
learning by clustering the data into 3 clusters based on their similarity and lastly
providing recommendations to the relevant stakeholders. By looking at the location data
and Foursquare data we explored, we could find out that the neighborhoods where a higher number of bakeries already exsit, mainly Queen’s Park and Ontario Provicial Governement. 

We also took at a look at the neighborhood that has other dessert shops and coffee shops. There are higher number of dessert shops in Christie, St. James Town, and Cabbagetown, and a number of coffee shops in Church and Wellesley, Kensington Market, Chinatown, and Grange Park. 

Considering that Celeste is trying to avoid opening the patisserie in a neighborhood that also has too many other bakeries, but needing to be where there are other business indicating a viable business location, it would seem that perhaps Christie, St. James Town, and Cabbagetown, could be good possibilies for a new patisserie since there are not many bakeries in these areas but the existence of dessert shops could mean a popular destination for this type of business. 